# Integrações com o FooDB - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando Tabelas dos dados originais do FooDB

Em relação ao modelo original proposto na etapa anterior, optou-se por montar mais relações a partir do id do que a partir do nome. A principal dificuldade encontrada para associar itens pelo nome é que alguns nomes podem chegar a quase 300 caracteres (principalmente de componentes químicos). O id, entretando, é um número menor e evitou a cópia de dados muito extensos a todo momento entre tabelas. Então, é importante ajustar os dados para refletir essas alterações.

### Classificação dos ingredientes

As categorias dos ingredientes não estão normalizadas nos dados do FooDB. Assim, primeiro elas precisam ser importadas diretamente da lista de ingredientes para depois serem ajustadas segundo o modelo proposto.

Importando os dados das classificações dos ingredientes para processar. 

In [2]:
DROP TABLE IF EXISTS Classificacao;

CREATE TABLE Classificacao (
  Id INT,
  CategoriaInferior VARCHAR(100),
  CategoriaSuperior VARCHAR(100),
  PRIMARY KEY (Id)
) AS SELECT
  id, food_subgroup, food_group
FROM CSVREAD('../data/raw/foodb/Food.csv');


Visualizando:

In [3]:
SELECT * FROM Classificacao LIMIT 10;


Aqui, percebeu-se que um dos subgrupos não está padronizado. Há duas categorias superiores "Herbs and spices" e "Herbs and Spices" utilizadas para uma mesma categoria. Esse é um dos problemas de essas informações estarem apenas junto dos ingredientes. Atualizando os valores inconsistentes:

In [4]:
UPDATE Classificacao C
SET C.CategoriaSuperior = 'Herbs and Spices'
WHERE C.CategoriaSuperior = 'Herbs and spices';


Criando uma VIEW para representar as relações entre os grupos e subgrupos:

In [5]:
DROP VIEW IF EXISTS GrupoSubgrupo;

CREATE VIEW GrupoSubgrupo AS
    SELECT DISTINCT CategoriaInferior, CategoriaSuperior
    FROM Classificacao
    WHERE CategoriaInferior <> ''
    ORDER BY CategoriaInferior, CategoriaSuperior;


Visualizando:

In [6]:
SELECT * FROM GrupoSubgrupo LIMIT 10;


Exportando esses dados:

In [7]:
CALL CSVWRITE('../data/processed/database/classificacao.csv',
              'SELECT * FROM GrupoSubgrupo');


119

### Dados dos ingredientes

Importando os dados dos ingredientes do FooDB.

In [8]:
DROP TABLE IF EXISTS Ingrediente;

CREATE TABLE Ingrediente (
  Id INT,
  Nome VARCHAR(100),
  Classificacao VARCHAR(100),
  PRIMARY KEY (Id),
) AS SELECT
  id, name, food_subgroup
FROM CSVREAD('../data/raw/foodb/Food.csv');


Adicionando novos ingredientes (serão usados para as outras bases). Os ids são continuações dos ids dos ingredientes do FooDB para não haver problemas de integração.

In [9]:
INSERT INTO Ingrediente VALUES (1025, 'Plant', NULL);
INSERT INTO Ingrediente VALUES (1026, 'Meat', NULL);
INSERT INTO Ingrediente VALUES (1027, 'Unknown', NULL);
INSERT INTO Ingrediente VALUES (1028, 'Essential oil', NULL);


Visualizando os dados:

In [10]:
SELECT * FROM Ingrediente LIMIT 10;


Esses são os dados necessários dos ingredientes. Exportando eles ordenados:

In [11]:
CALL CSVWRITE('../data/interim/foodb/ingredientes.csv',
              'SELECT * FROM Ingrediente ORDER BY Id');


996

### Componentes (componentes químicos e macronutrientes)

Os componentes químicos e macronutrientes estão em tabelas diferentes no FooDB. Assim, para seguir o modelo proposto, primeiro eles serão importados e depois serão associados em uma única tabela com um atributo de tipo. Esse atributo pode ser preenchido durante a importação dos dados:

In [12]:
DROP TABLE IF EXISTS Nutrientes;
DROP TABLE IF EXISTS Componentes;

CREATE TABLE Nutrientes (
  Id INT,
  Nome VARCHAR(300),
  Tipo VARCHAR(20),
  PRIMARY KEY (Id)
) AS SELECT
  id, name, 'Nutriente'
FROM CSVREAD('../data/raw/foodb/Nutrient.csv');

CREATE TABLE Componentes (
  Id INT,
  Nome VARCHAR(300),
  Tipo VARCHAR(20),
  PRIMARY KEY (Id)
) AS SELECT
  id, name, 'Componente'
FROM CSVREAD('../data/raw/foodb/Compound.csv');


Visualizando os dados:

In [13]:
SELECT * FROM Nutrientes LIMIT 10;


In [14]:
SELECT * FROM Componentes LIMIT 10;


Agora, é possível realizar a união:

In [15]:
DROP VIEW IF EXISTS UniaoComponentesNutrientes;

CREATE VIEW UniaoComponentesNutrientes AS
(SELECT * FROM Nutrientes
UNION
SELECT * FROM Componentes)
ORDER BY Id;


Visualizando os dados. É claro que, ao realizar a união, há ids repetidos. Logo, a chave primária dessa nova tabela deve ser considerada como id + tipo, assim como foi proposto.

In [16]:
SELECT * FROM UniaoComponentesNutrientes LIMIT 10;


Exportando a VIEW diretamente para os dados da base final:

In [17]:
CALL CSVWRITE('../data/processed/database/componente.csv',
              'SELECT * FROM UniaoComponentesNutrientes');


70516

### Quantidades dos componentes nos ingredientes

Por fim, falta relacionar os ingredientes com as quantidades de componentes deles. Os valores médios serão considerados a partir dos dados do FooDB. 

In [18]:
DROP TABLE IF EXISTS Associacao;

CREATE TABLE Associacao (
  Id INT,
  IdComponente INT,
  TipoComponente VARCHAR(20),
  FoodId INT,
  Media DECIMAL(10, 3),
  Unidade VARCHAR(30),
  PRIMARY KEY (Id)
) AS SELECT
  id, source_id, source_type, food_id, orig_content, orig_unit
FROM CSVREAD('../data/raw/foodb/Content.csv');


Visualizando os dados:

In [19]:
SELECT * FROM Associacao LIMIT 10;


Os dados contém informações sobre diferentes partes/tipos dos alimentos e as quantidades dos componentes. Então, é possível agrupar ingredientes, componentes e unidades de forma única e fazer a média dos valores. Isso fornece dados únicos para cada ingrediente.

Um problema enfrentado nesse caso foi a grande quantidade de dados. Há mais de 70 mil componentes existentes, mas muitos ingredientes têm quantidades de componentes não quantificadas. Assim, para simplificação - e por as análises serem feitas com quantidades quantificáveis - apenas componentes com quantidade definida são considerados.

In [20]:
DROP VIEW IF EXISTS QuantidadesPadronizadas;

CREATE VIEW QuantidadesPadronizadas AS
SELECT I.Nome, A.IdComponente, A.TipoComponente, AVG(A.Media) Quantidade, A.Unidade
FROM Associacao A, Ingrediente I
WHERE A.Unidade <> '' AND A.FoodId = I.Id
GROUP BY I.Nome, A.IdComponente, A.TipoComponente, A.Unidade
HAVING Quantidade > 0;


Visualizando as quantidades padronizadas:

In [21]:
SELECT * FROM QuantidadesPadronizadas LIMIT 10;


Resta agora associar as quantidades aos ingredientes novos criados. 

Esses ingredientes estão presentes para lidar com a ausência de alguns ingredientes do FooDB. Ao associar as receitas de várias bases, alguns ingredientes não catalogados foram identificados e algumas associações precisaram ser feitas.

Os 4 ingredientes novos criados (sem considerar ingredientes compostos ainda) são "Plant", "Meat", "Unknown" e "Essential oil". "Plant" foi utilizado quando alguma planta não catalogada foi encontrada. "Meat" foi utilizado quando algum tipo de carne não catalogado foi encontrado. "Unknown" se refere principalmente a produtos de cozimento (spray de cozimento, corante alimentício, ...) e "Essential oil" foi utilizado para alguns óleos de plantas muito específicas.

A partir de pesquisas, os dois últimos não contêm valor nutricional. Entretanto, "Plant" e "Meat" têm um valor associado. As opções encontradas então foram: buscar por dados em outras bases, calcular os valores com base em dados disponíveis de outros ingredientes ou apenas ignorar as composições nutricionais.

Inicialmente foram buscados dados em outras bases e conteúdos disponíveis. O principal problema, nesse caso, é que tipos diferentes de alimentos mesmo dentro dessas categorias têm composições muito diferentes. Então, tanto essa solução quanto tentar calcular dados não foi viável, porque prezou-se pela consistência de dados reais. Assim, esses 4 ingredientes não têm nenhum valor nutricional agregado.

Um questionamento possível nesse caso é "então por que esses ingredientes existem?". Apesar de serem de certa forma generalizados, esses ingredientes permitem uma análise mais rica das receitas. Caso eles não existissem, não seria possível associar ingredientes de algumas receitas a nenhum dado na base, o que significaria que elas seriam desconsideradas. Assim, a criação desses ingredientes, apesar de não contribuir com análises de composição, possibilita observar melhor relações de outros ingredientes. Mais detalhes estão disponíveis em outros notebooks.

Esses valores então já podem ser exportados. Exportando a VIEW:

In [22]:
CALL CSVWRITE('../data/interim/foodb/componentesdosingredientes.csv',
              'SELECT * FROM QuantidadesPadronizadas');


75790